<h1>Aggregate all 30 stocks in the DJIA</h1>

In [163]:
DJIA_STOCKS = ['MSFT', 'AAPL', 'AMZN', 'JPM', 'WMT', 'V', 'UNH', 'PG', 'JNJ', 'HD', 'MRK', 'CVX', 'KO', 'CRM', 'CSCO', 'MCD', 'DIS', 'VZ', 'AXP', 'AMGN', 'CAT', 'IBM', 'GS', 'HON', 'INTC', 'NIKE', 'BA', 'MMM', 'TRV', 'DOW']
len(DJIA_STOCKS) == 30
# No pruning of data needed.

True

<h2>Yahoo Finance does not require an api key!</h2>
<b>Run this cell once, then all of the data will be saved in a pickle (and csv files)!</b>

In [164]:
import yfinance as yf
import pickle
import os

stock_market_cap = {}
stock_data = {}

if 'DJIA Historical Data' not in os.listdir():
    os.mkdir('DJIA Historical Data')

    for stock in DJIA_STOCKS:
        ticker_data = yf.Ticker(stock)
        hist = ticker_data.history(period='max')
    
        hist.to_csv(f'DJIA Historical Data/{stock}.csv')
        stock_data[stock] = hist
        if 'marketCap' in ticker_data.info:
            stock_market_cap[stock] = ticker_data.info['marketCap']
        else:
            print(f'{stock} DOES NOT HAVE MARKET CAP DATA.')
else:
    print("DJIA Historical Data already pulled.")


if 'stock_ohlcv.pkl' not in os.listdir():
    pickle.dump(stock_data, open('stock_ohlcv.pkl', 'wb'))
else:
    print("DJIA's OHLCV Data already pickled.")


if 'stock_market_cap.pkl' not in os.listdir():
    pickle.dump(stock_market_cap, open('stock_market_cap.pkl', 'wb'))
else:
    print("DJIA's Market Cap Data already pickled.")

print("DONE!")

DJIA Historical Data already pulled.
DJIA's OHLCV Data already pickled.
DJIA's Market Cap Data already pickled.
DONE!


<h1>Once you download all of the DJIA data, always load the data here!</h1>

In [165]:
djia_data = pickle.load(open('stock_ohlcv.pkl', 'rb'))
djia_market_cap = pickle.load(open('stock_market_cap.pkl', 'rb'))

<h1>Remove all stocks with no entries.</h1>

In [166]:
for stock in DJIA_STOCKS:
    if len(djia_data[stock]) <= 0:
        DJIA_STOCKS.remove(stock)
print(f'STOCKS IN DJIA CURRENTLY: {len(DJIA_STOCKS)}')

STOCKS IN DJIA CURRENTLY: 29


<H1>Calculate 1M, 6M, 2Y, and ALL-TIME volatility, average price, increasing days, and decreasing days.</H1>

In [167]:
import numpy as np
import datetime as dt
from pandas import DatetimeIndex
today = np.datetime64('2024-07-06', 'ns')

In [199]:
import pandas as pd
pd.set_option('display.width', 200) 

temp_djia_data = {}
summary_statistics = pd.DataFrame(index=DJIA_STOCKS)

#temp = []
for stock in DJIA_STOCKS:
    df = djia_data[stock]
    df.index = df.index.tz_localize(None)
    df = df.reset_index()
    temp_djia_data[stock] = df
    #temp_df = df[df['Date'] > today - np.timedelta64(7, 'D')]
    #temp_df = temp_df.reset_index()
    #print(temp_df)
    #df[date] = df[df['Date'] > today]
    #print(df)
temp_stats = {}
for stock in DJIA_STOCKS: #1M Volatility
    df = temp_djia_data[stock]
    df = df[df['Date'] > today - np.timedelta64(31, 'D')]
    std = df['Close'].std()
    count = df['Close'].count()
    temp_stats[stock] = std / math.sqrt(count)
summary_statistics['1M Vol.'] = temp_stats
temp_stats = {}

for stock in DJIA_STOCKS: # 1M Average Price
    df = temp_djia_data[stock]
    df = df[df['Date'] > today - np.timedelta64(31, 'D')]

    avg = df['Close'].mean()
    temp_stats[stock] = avg
summary_statistics['1M Avg. Close'] = temp_stats
temp_stats = {}



temp_stats1 = {}
for stock in DJIA_STOCKS: #1M Increasing & Decreasing Days
    df = temp_djia_data[stock]
    df = df[df['Date'] > today - np.timedelta64(31, 'D')]
    inc = dec = 0

    skipFirst = False
    last_price = None
    for close in df['Close'].iloc:
        if not skipFirst:
            skipFirst = True
            last_price = close
            continue
        if last_price > close:
            dec += 1
        else:
            inc += 1
    temp_stats[stock] = inc
    temp_stats1[stock] = dec
summary_statistics['1M Inc. Days'] = temp_stats
summary_statistics['1M Dec. Days'] = temp_stats1

temp_stats = {}
for stock in DJIA_STOCKS: #6M Volatility
    df = temp_djia_data[stock]
    df = df[df['Date'] > today - np.timedelta64(31*6, 'D')]
    std = df['Close'].std()
    count = df['Close'].count()
    temp_stats[stock] = std / math.sqrt(count)
summary_statistics['6M Vol.'] = temp_stats
temp_stats = {}

for stock in DJIA_STOCKS: # 6M Average Price
    df = temp_djia_data[stock]
    df = df[df['Date'] > today - np.timedelta64(31*6, 'D')]

    avg = df['Close'].mean()
    temp_stats[stock] = avg
summary_statistics['6M Avg. Close'] = temp_stats
temp_stats = {}

temp_stats1 = {}
for stock in DJIA_STOCKS: #6M Increasing & Decreasing Days
    df = temp_djia_data[stock]
    df = df[df['Date'] > today - np.timedelta64(31*6, 'D')]
    inc = dec = 0

    skipFirst = False
    last_price = None
    for close in df['Close'].iloc:
        if not skipFirst:
            skipFirst = True
            last_price = close
            continue
        if last_price > close:
            dec += 1
        else:
            inc += 1
        last_price = close
    temp_stats[stock] = inc
    temp_stats1[stock] = dec
summary_statistics['6M Inc. Days'] = temp_stats
summary_statistics['6M Dec. Days'] = temp_stats1



temp_stats = {}
for stock in DJIA_STOCKS: #2Y Volatility
    df = temp_djia_data[stock]
    df = df[df['Date'] > today - np.timedelta64(365*2, 'D')]
    std = df['Close'].std()
    count = df['Close'].count()
    temp_stats[stock] = std / math.sqrt(count)
summary_statistics['2Y Vol.'] = temp_stats
temp_stats = {}

for stock in DJIA_STOCKS: # 2Y Average Price
    df = temp_djia_data[stock]
    df = df[df['Date'] > today - np.timedelta64(365*2, 'D')]

    avg = df['Close'].mean()
    temp_stats[stock] = avg
summary_statistics['2Y Avg. Close'] = temp_stats
temp_stats = {}

temp_stats1 = {}
for stock in DJIA_STOCKS: #6M Increasing & Decreasing Days
    df = temp_djia_data[stock]
    df = df[df['Date'] > today - np.timedelta64(365*2, 'D')]
    inc = dec = 0

    skipFirst = False
    last_price = None
    for close in df['Close'].iloc:
        if not skipFirst:
            skipFirst = True
            last_price = close
            continue
        if last_price > close:
            dec += 1
        else:
            inc += 1
        last_price = close
    temp_stats[stock] = inc
    temp_stats1[stock] = dec
summary_statistics['2Y Inc. Days'] = temp_stats
summary_statistics['2Y Dec. Days'] = temp_stats1




temp_stats = {}
for stock in DJIA_STOCKS: #ALL-TIME Volatility
    df = temp_djia_data[stock]
    std = df['Close'].std()
    count = df['Close'].count()
    temp_stats[stock] = std / math.sqrt(count)
summary_statistics['ALL-TIME Vol.'] = temp_stats
temp_stats = {}

for stock in DJIA_STOCKS: # ALL-TIME Average Price
    df = temp_djia_data[stock]

    avg = df['Close'].mean()
    temp_stats[stock] = avg
summary_statistics['ALL-TIME Avg. Close'] = temp_stats
temp_stats = {}

temp_stats1 = {}
for stock in DJIA_STOCKS: #ALL-TIME Increasing & Decreasing Days
    df = temp_djia_data[stock]
    inc = dec = 0

    skipFirst = False
    last_price = None
    for close in df['Close'].iloc:
        if not skipFirst:
            skipFirst = True
            last_price = close
            continue
        if last_price > close:
            dec += 1
        else:
            inc += 1
        last_price = close
    temp_stats[stock] = inc
    temp_stats1[stock] = dec
summary_statistics['ALL-TIME Inc. Days'] = temp_stats
summary_statistics['ALL-TIME Dec. Days'] = temp_stats1

print(summary_statistics)

       1M Vol.  1M Avg. Close  1M Inc. Days  1M Dec. Days   6M Vol.  6M Avg. Close  6M Inc. Days  6M Dec. Days   2Y Vol.  2Y Avg. Close  2Y Inc. Days  2Y Dec. Days  ALL-TIME Vol.  \
MSFT  2.626514     445.962001            18             1  1.708001     414.842490            70            56  3.036538     322.254734           261           239       0.903777   
AAPL  1.900753     210.983000            18             1  1.261706     185.132007            67            59  0.894156     169.493257           263           237       0.428827   
AMZN  1.346333     189.571500            14             5  1.066291     176.382598            69            57  1.372040     133.238743           261           239       0.644531   
JPM   1.094499     197.975735            13             6  1.010656     187.254909            72            54  1.257126     145.901219           280           220       0.387759   
WMT   0.207225      67.586000            13             6  0.406365      60.246428        

<h1>Calculating returns</h1>

In [ ]:
djia_returns = {}

for stock in DJIA_STOCKS:
    